<a href="https://colab.research.google.com/github/keripikkaneboo/Hands-On-Machine-Learning-O-Reilly-/blob/main/16.%20Chapter16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bab 16: Natural Language Processing with RNN and Attention

Bab ini melanjutkan pembahasan tentang RNN dan menunjukkan bagaimana mereka dapat diterapkan pada tugas-tugas **Natural Language Processing (NLP)**. Selain itu, bab ini memperkenalkan **attention mechanisms** dan arsitektur **Transformer** yang revolusioner.

* **Text Generation dengan Character-RNN**:
    * **Konsep**: Melatih RNN untuk memprediksi karakter berikutnya dalam sebuah urutan teks. Setelah dilatih, model ini dapat digunakan untuk menghasilkan teks baru, satu karakter pada satu waktu.
    * **Dataset**: Bab ini menunjukkan cara mempersiapkan dataset dari teks mentah (seperti karya Shakespeare) dengan mengubahnya menjadi jendela-jendela teks yang tumpang tindih (*overlapping windows*) menggunakan `tf.data`.
    * **Stateful RNN**: Diperkenalkan konsep RNN *stateful*, yang mempertahankan *hidden state*-nya di antara *batch* training. Ini memungkinkannya untuk belajar pola jangka panjang, tetapi memerlukan persiapan data yang lebih hati-hati (urutan data harus berkesinambungan).

* **Analisis Sentimen**:
    * Beralih dari level karakter ke level kata. Model dilatih untuk mengklasifikasikan teks (misalnya, ulasan film) sebagai positif atau negatif.
    * **Tokenisasi**: Proses memecah teks menjadi kata-kata atau *subword*. Bab ini menyinggung tantangan tokenisasi untuk berbagai bahasa dan memperkenalkan teknik *subword* modern.
    * **Word Embeddings**: Kata-kata direpresentasikan sebagai vektor padat (*dense vector*) yang dapat dilatih. Lapisan `keras.layers.Embedding` digunakan untuk ini.
    * **Masking**: Teknik penting untuk menangani urutan dengan panjang yang bervariasi. Dengan menyetel `mask_zero=True` pada lapisan *embedding*, token *padding* (biasanya dengan ID 0) akan diabaikan oleh lapisan-lapisan berikutnya.
    * **Menggunakan Pretrained Embeddings**: Untuk meningkatkan performa, terutama dengan data yang sedikit, kita dapat menggunakan *word embeddings* yang sudah dilatih pada korpus teks yang besar (misalnya, dari TensorFlow Hub).

* **Neural Machine Translation (NMT)**:
    * **Arsitektur Encoder-Decoder**: Arsitektur standar untuk NMT. *Encoder* (RNN) membaca kalimat sumber dan merangkumnya menjadi sebuah vektor konteks (*hidden state* terakhir). *Decoder* (RNN lain) kemudian menggunakan vektor konteks ini untuk menghasilkan terjemahan, kata demi kata.
    * **Attention Mechanisms**: Solusi untuk masalah memori jangka panjang pada arsitektur Encoder-Decoder. *Attention* memungkinkan *decoder* untuk fokus pada bagian-bagian yang relevan dari kalimat sumber saat menghasilkan setiap kata terjemahan. Ini secara drastis meningkatkan kualitas terjemahan, terutama untuk kalimat panjang.

* **Arsitektur Transformer**:
    * Diperkenalkan dalam paper "Attention Is All You Need", arsitektur ini sepenuhnya meninggalkan lapisan rekuren dan konvolusional, dan hanya mengandalkan *attention mechanisms*.
    * **Positional Embeddings**: Karena model ini tidak memiliki sifat rekuren, ia tidak memiliki informasi urutan. *Positional embeddings* ditambahkan ke *word embeddings* untuk memberikan informasi posisi absolut dan relatif dari setiap kata.
    * **Multi-Head Attention**: Komponen inti dari Transformer. Ia memungkinkan model untuk secara bersamaan memperhatikan informasi dari *subspace* representasi yang berbeda. Ini terdiri dari beberapa *Scaled Dot-Product Attention* yang bekerja secara paralel.

### 1. Menghasilkan Teks dengan Character-RNN
Contoh ini melatih model untuk menghasilkan teks yang mirip tulisan Shakespeare.

```python
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Mengunduh dataset karya Shakespeare
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

# Membuat tokenizer tingkat karakter
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

# Mengubah teks menjadi urutan ID karakter
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
max_id = len(tokenizer.word_index)

# Mempersiapkan dataset
train_size = int(len(encoded) * 0.9)
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.shuffle(10000).batch(32)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

# Membangun model Char-RNN
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
# history = model.fit(dataset, epochs=10) # Training bisa memakan waktu lama

# Fungsi untuk menghasilkan teks
def complete_text(text, n_chars=50, temperature=1):
    def preprocess(texts):
        X = np.array(tokenizer.texts_to_sequences(texts)) - 1
        return tf.one_hot(X, max_id)
    def next_char(text, temperature=1):
        X_new = preprocess([text])
        y_proba = model.predict(X_new)[0, -1:, :]
        rescaled_logits = tf.math.log(y_proba) / temperature
        char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
        return tokenizer.sequences_to_texts(char_id.numpy())[0]

    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

# print(complete_text("To be or not to b", temperature=0.5)) # Jalankan setelah melatih model
print("Model Char-RNN telah dibuat. Hapus komentar pada `model.fit` untuk melatihnya.")
```

### 2. Analisis Sentimen dengan Embedding dan Masking
Model ini mengklasifikasikan ulasan film dari dataset IMDb.

```python
# Diperlukan untuk Google Colab
!pip install -q -U tensorflow_datasets

import tensorflow_datasets as tfds

# Memuat dataset imdb_reviews
datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_set = datasets["train"].batch(32).prefetch(1)
valid_set = datasets["test"].take(2500).batch(32).prefetch(1) # Ambil sebagian untuk validasi
test_set = datasets["test"].skip(2500).batch(32).prefetch(1)

# Membuat dan mengadaptasi lapisan TextVectorization
vocab_size = 1000
text_vec_layer = keras.layers.TextVectorization(max_tokens=vocab_size)
text_vec_layer.adapt(train_set.map(lambda review, label: review))

# Membangun model dengan Embedding dan Masking
embed_size = 128
model = keras.models.Sequential([
    text_vec_layer,
    # mask_zero=True akan mengabaikan token padding (ID=0)
    keras.layers.Embedding(vocab_size, embed_size, mask_zero=True),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# history = model.fit(train_set, validation_data=valid_set, epochs=5)
print("\nModel analisis sentimen telah dibuat.")
```

### 3. Arsitektur Transformer (Konseptual)
Transformer tidak lagi menggunakan RNN, melainkan murni *attention*. Komponen kuncinya adalah **Positional Encoding** dan **Multi-Head Attention**.

```python
# --- Lapisan Positional Encoding Kustom ---
class PositionalEncoding(keras.layers.Layer):
    def __init__(self, max_steps, max_dims, dtype=tf.float32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        if max_dims % 2 == 1: max_dims += 1
        p, i = np.meshgrid(np.arange(max_steps), np.arange(max_dims // 2))
        pos_emb = np.empty((1, max_steps, max_dims))
        pos_emb[0, :, ::2] = np.sin(p / 10000**(2 * i / max_dims)).T
        pos_emb[0, :, 1::2] = np.cos(p / 10000**(2 * i / max_dims)).T
        self.positional_embedding = tf.constant(pos_emb.astype(self.dtype))
    def call(self, inputs):
        shape = tf.shape(inputs)
        return inputs + self.positional_embedding[:, :shape[-2], :shape[-1]]

# --- Lapisan Multi-Head Attention Sederhana ---
# Ini adalah versi konseptual, implementasi sebenarnya lebih kompleks
class MultiHeadAttention(keras.layers.Layer):
    def __init__(self, n_heads, causal=False, **kwargs):
        super().__init__(**kwargs)
        self.n_heads = n_heads
        self.causal = causal
        # Lapisan-lapisan ini akan dibuat di dalam metode build()
        # self.q_dense, self.k_dense, self.v_dense, self.attention, self.out_dense
    def build(self, batch_input_shape):
      # Implementasi build
      pass
    def call(self, inputs, mask=None):
      # Logika call
      pass

# Contoh kerangka model Transformer Encoder
embed_size = 128
max_steps = 500
vocab_size = 10000

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
embeddings = keras.layers.Embedding(vocab_size, embed_size)(encoder_inputs)
encoder_in = PositionalEncoding(max_steps, max_dims=embed_size)(embeddings)

# Z = encoder_in
# for N in range(6): # Tumpukan 6 blok encoder
#    Z = MultiHeadAttention(...)(Z) # Blok attention
#    Z = keras.layers.LayerNormalization()(Z)
#    ... # Feed Forward Network
# encoder_outputs = Z
print("\nKerangka konseptual untuk Transformer telah disiapkan.")
```

